# Large Scale Distributed Seismic Processing - Poseidon Dataset

This is a workflow where we will work through performing calculations on post-stack seismic data using cloud computing.

We will use this Jupyter notebook to walk through the steps, S3 to store the data, and Lambda to do the processing.

The seismic data is streamed from S3 without the need to duplicate any portion of it. This is ideal for situations where you need to work with large (+100GB) SEGY files that cannot always be loaded into memory and to also avoid the cost of making a duplicate on a local filesystem. S3 allows you to read specific bytes of data from the file allowing for reading and processing of the traces you need. Lambda will allow you to parallel process the data and with no need to provision any hardware, you only pay for the time it takes to perform the calculations.


### Import packages

In [ ]:
import sys
import time
import json
import boto3
import struct
import pickle
import botocore
import array as arr
import numpy as np
import matplotlib.pyplot as plt
from struct import Struct

### Functions

Below are some custom fuctions that load SEGY revision 1 data.  They will aid in decifering the SEGY format.

In [ ]:
# Decode the text header block
def DecodeTextHeader(text_header_raw):
    text_header = text_header_raw.decode('cp500')
    text_header = text_header.replace("C 1 ", "\nC 1 ")
    text_header = text_header.replace("C 2 ", "\nC 2 ")
    text_header = text_header.replace("C 3 ", "\nC 3 ")
    text_header = text_header.replace("C 4 ", "\nC 4 ")
    text_header = text_header.replace("C 5 ", "\nC 5 ")
    text_header = text_header.replace("C 6 ", "\nC 6 ")
    text_header = text_header.replace("C 7 ", "\nC 7 ")
    text_header = text_header.replace("C 8 ", "\nC 8 ")
    text_header = text_header.replace("C 9 ", "\nC 9 ")
    text_header = text_header.replace("C10 ", "\nC10 ")
    text_header = text_header.replace("C11 ", "\nC11 ")
    text_header = text_header.replace("C12 ", "\nC12 ")
    text_header = text_header.replace("C13 ", "\nC13 ")
    text_header = text_header.replace("C14 ", "\nC14 ")
    text_header = text_header.replace("C15 ", "\nC15 ")
    text_header = text_header.replace("C16 ", "\nC16 ")
    text_header = text_header.replace("C17 ", "\nC17 ")
    text_header = text_header.replace("C18 ", "\nC18 ")
    text_header = text_header.replace("C19 ", "\nC19 ")
    text_header = text_header.replace("C20 ", "\nC20 ")
    text_header = text_header.replace("C21 ", "\nC21 ")
    text_header = text_header.replace("C22 ", "\nC22 ")
    text_header = text_header.replace("C23 ", "\nC23 ")
    text_header = text_header.replace("C24 ", "\nC24 ")
    text_header = text_header.replace("C25 ", "\nC25 ")
    text_header = text_header.replace("C26 ", "\nC26 ")
    text_header = text_header.replace("C27 ", "\nC27 ")
    text_header = text_header.replace("C28 ", "\nC28 ")
    text_header = text_header.replace("C29 ", "\nC29 ")
    text_header = text_header.replace("C30 ", "\nC30 ")
    text_header = text_header.replace("C31 ", "\nC31 ")
    text_header = text_header.replace("C32 ", "\nC32 ")
    text_header = text_header.replace("C33 ", "\nC33 ")
    text_header = text_header.replace("C34 ", "\nC34 ")
    text_header = text_header.replace("C35 ", "\nC35 ")
    text_header = text_header.replace("C36 ", "\nC36 ")
    text_header = text_header.replace("C37 ", "\nC37 ")
    text_header = text_header.replace("C38 ", "\nC38 ")
    text_header = text_header.replace("C39 ", "\nC39 ")
    text_header = text_header.replace("C40 ", "\nC40 ")
    
    return text_header

# Decode the binary header block
def DecodeBinHeader(bin_header_raw):
    bin_header = {}

    bin_header['job_id']                  = int.from_bytes(bin_header_raw[0:4], byteorder='big', signed=False)
    bin_header['line_no']                 = int.from_bytes(bin_header_raw[4:8], byteorder='big', signed=False)
    bin_header['reel_no']                 = int.from_bytes(bin_header_raw[8:12], byteorder='big', signed=False)
    bin_header['data_traces']             = int.from_bytes(bin_header_raw[12:14], byteorder='big', signed=False)
    bin_header['aux_traces']              = int.from_bytes(bin_header_raw[14:16], byteorder='big', signed=False)
    bin_header['sample_interval']         = int.from_bytes(bin_header_raw[16:18], byteorder='big', signed=False)
    bin_header['sample_interval_orig']    = int.from_bytes(bin_header_raw[18:20], byteorder='big', signed=False)
    bin_header['samples_per_trace']       = int.from_bytes(bin_header_raw[20:22], byteorder='big', signed=False)
    bin_header['samples_per_trace_orig']  = int.from_bytes(bin_header_raw[22:24], byteorder='big', signed=False)
    bin_header['data_sample_format']      = int.from_bytes(bin_header_raw[24:26], byteorder='big', signed=False)
    bin_header['ensemble_fold']           = int.from_bytes(bin_header_raw[26:28], byteorder='big', signed=False)
    bin_header['trace_sorting']           = int.from_bytes(bin_header_raw[28:30], byteorder='big', signed=False)
    bin_header['vert_sum_code']           = int.from_bytes(bin_header_raw[30:32], byteorder='big', signed=False)
    bin_header['sweep_hz_start']          = int.from_bytes(bin_header_raw[32:34], byteorder='big', signed=False)
    bin_header['sweep_hz_end']            = int.from_bytes(bin_header_raw[34:36], byteorder='big', signed=False)
    bin_header['sweep_length']            = int.from_bytes(bin_header_raw[36:38], byteorder='big', signed=False)
    bin_header['sweep_type']              = int.from_bytes(bin_header_raw[38:40], byteorder='big', signed=False)
    bin_header['sweep_trace_ch']          = int.from_bytes(bin_header_raw[40:42], byteorder='big', signed=False)
    bin_header['sweep_trace_taper_start'] = int.from_bytes(bin_header_raw[42:44], byteorder='big', signed=False)
    bin_header['sweep_trace_taper_end']   = int.from_bytes(bin_header_raw[44:46], byteorder='big', signed=False)
    bin_header['taper_type']              = int.from_bytes(bin_header_raw[46:48], byteorder='big', signed=False)
    bin_header['data_traces_correlated']  = int.from_bytes(bin_header_raw[48:50], byteorder='big', signed=False)
    bin_header['binary_gain_recovered']   = int.from_bytes(bin_header_raw[50:52], byteorder='big', signed=False)
    bin_header['amp_recovery_method']     = int.from_bytes(bin_header_raw[52:54], byteorder='big', signed=False)
    bin_header['measurement_system']      = int.from_bytes(bin_header_raw[54:56], byteorder='big', signed=False)
    bin_header['impulse_sig_polarity']    = int.from_bytes(bin_header_raw[56:58], byteorder='big', signed=False)
    bin_header['vib_polarity']            = int.from_bytes(bin_header_raw[58:60], byteorder='big', signed=False)
    bin_header['unassigned']              = int.from_bytes(bin_header_raw[60:300], byteorder='big', signed=False)
    bin_header['segy_format']             = int.from_bytes(bin_header_raw[300:302], byteorder='big', signed=False)
    bin_header['fixed_length_flag']       = int.from_bytes(bin_header_raw[302:304], byteorder='big', signed=False)
    bin_header['extended_text_header_no'] = int.from_bytes(bin_header_raw[304:306], byteorder='big', signed=False)
    bin_header['unassigned2']             = int.from_bytes(bin_header_raw[306:400], byteorder='big', signed=False)
    
    return bin_header

# Display the binary header data.
def PrintBinHeader(bin_header):
    print("job_id                  = ", bin_header['job_id']                 )
    print("line_no                 = ", bin_header['line_no']                )
    print("reel_no                 = ", bin_header['reel_no']                )
    print("data_traces             = ", bin_header['data_traces']            )
    print("aux_traces              = ", bin_header['aux_traces']             )
    print("sample_interval         = ", bin_header['sample_interval']        )
    print("sample_interval_orig    = ", bin_header['sample_interval_orig']   )
    print("samples_per_trace       = ", bin_header['samples_per_trace']      )
    print("samples_per_trace_orig  = ", bin_header['samples_per_trace_orig'] )
    print("data_sample_format      = ", bin_header['data_sample_format']     )
    print("ensemble_fold           = ", bin_header['ensemble_fold']          )
    print("trace_sorting           = ", bin_header['trace_sorting']          )
    print("vert_sum_code           = ", bin_header['vert_sum_code']          )
    print("sweep_hz_start          = ", bin_header['sweep_hz_start']         )
    print("sweep_hz_end            = ", bin_header['sweep_hz_end']           )
    print("sweep_length            = ", bin_header['sweep_length']           )
    print("sweep_type              = ", bin_header['sweep_type']             )
    print("sweep_trace_ch          = ", bin_header['sweep_trace_ch']         )
    print("sweep_trace_taper_start = ", bin_header['sweep_trace_taper_start'])
    print("sweep_trace_taper_end   = ", bin_header['sweep_trace_taper_end']  )
    print("taper_type              = ", bin_header['taper_type']             )
    print("data_traces_correlated  = ", bin_header['data_traces_correlated'] )
    print("binary_gain_recovered   = ", bin_header['binary_gain_recovered']  )
    print("amp_recovery_method     = ", bin_header['amp_recovery_method']    )
    print("measurement_system      = ", bin_header['measurement_system']     )
    print("impulse_sig_polarity    = ", bin_header['impulse_sig_polarity']   )
    print("vib_polarity            = ", bin_header['vib_polarity']           )
    print("unassigned              = ", bin_header['unassigned']             )
    print("segy_format             = ", bin(bin_header['segy_format'])[2:]   )    
    print("fixed_length_flag       = ", bin_header['fixed_length_flag']      )
    print("extended_text_header_no = ", bin_header['extended_text_header_no'])
    print("unassigned2             = ", bin_header['unassigned2']            )
    
# Decode a binary trade header.
def DecodeTraceHeader(trace_header_raw):
    trace_header = {}
    trace_header['trace_seq_no_all']            = int.from_bytes(trace_header_raw[0:4], byteorder='big', signed=False)
    trace_header['trace_seq_no_file']           = int.from_bytes(trace_header_raw[4:8], byteorder='big', signed=False)
    trace_header['field_record_no_orig']        = int.from_bytes(trace_header_raw[8:12], byteorder='big', signed=False)
    trace_header['trace_no_field_orig']         = int.from_bytes(trace_header_raw[12:16], byteorder='big', signed=False)
    trace_header['energy_source_point_no']      = int.from_bytes(trace_header_raw[16:20], byteorder='big', signed=False)
    trace_header['ensemble_no']                 = int.from_bytes(trace_header_raw[20:24], byteorder='big', signed=False)
    trace_header['ensemble_trace_no']           = int.from_bytes(trace_header_raw[24:28], byteorder='big', signed=False)
    trace_header['trace_id']                    = int.from_bytes(trace_header_raw[28:30], byteorder='big', signed=False)
    trace_header['sum_vertical_traces']         = int.from_bytes(trace_header_raw[30:32], byteorder='big', signed=False)
    trace_header['sum_horizontal_traces']       = int.from_bytes(trace_header_raw[32:34], byteorder='big', signed=False)
    trace_header['data_use']                    = int.from_bytes(trace_header_raw[34:36], byteorder='big', signed=False)
    trace_header['distance_from_source_center'] = int.from_bytes(trace_header_raw[36:40], byteorder='big', signed=False)
    # ... incomplete
    trace_header['group_x']                     = int.from_bytes(trace_header_raw[80:84], byteorder='big', signed=False)
    trace_header['group_y']                     = int.from_bytes(trace_header_raw[84:88], byteorder='big', signed=False)
    trace_header['coord_units']                 = int.from_bytes(trace_header_raw[88:90], byteorder='big', signed=False)
    trace_header['trace_samples']               = int.from_bytes(trace_header_raw[114:116], byteorder='big', signed=False)
    trace_header['sample_interval']             = int.from_bytes(trace_header_raw[116:118], byteorder='big', signed=False)
    trace_header['gain_type']                   = int.from_bytes(trace_header_raw[118:120], byteorder='big', signed=False)
    trace_header['CDP_X']                       = int.from_bytes(trace_header_raw[180:184], byteorder='big', signed=False)
    trace_header['CDP_Y']                       = int.from_bytes(trace_header_raw[184:188], byteorder='big', signed=False)
    trace_header['inline']                      = int.from_bytes(trace_header_raw[188:192], byteorder='big', signed=False)
    trace_header['xline']                       = int.from_bytes(trace_header_raw[192:196], byteorder='big', signed=False)
    trace_header['trace_unit']                  = int.from_bytes(trace_header_raw[202:204], byteorder='big', signed=False) 
    trace_header['inline_custom']               = int.from_bytes(trace_header_raw[232:236], byteorder='big', signed=False)
    trace_header['xline_custom']                = int.from_bytes(trace_header_raw[236:240], byteorder='big', signed=False)

    return trace_header
    
# Display the trade header data.
def PrintTraceHeaders(trace_header):
    print("trace_seq_no_all            = ", trace_header['trace_seq_no_all'])
    print("trace_seq_no_file           = ", trace_header['trace_seq_no_file'])
    print("field_record_no_orig        = ", trace_header['field_record_no_orig'])
    print("trace_no_field_orig         = ", trace_header['trace_no_field_orig'])
    print("energy_source_point_no      = ", trace_header['energy_source_point_no'])
    print("ensemble_no                 = ", trace_header['ensemble_no'])
    print("ensemble_trace_no           = ", trace_header['ensemble_trace_no'])
    print("trace_id                    = ", trace_header['trace_id'])
    print("sum_vertical_traces         = ", trace_header['sum_vertical_traces'])
    print("sum_horizontal_traces       = ", trace_header['sum_horizontal_traces'])
    print("data_use                    = ", trace_header['data_use'])
    print("distance_from_source_center = ", trace_header['distance_from_source_center'])
    # ... incomplete
    print("group_x                     = ", trace_header['group_x'])
    print("group_y                     = ", trace_header['group_y'])
    print("coord_units                 = ", trace_header['coord_units'])
    print("trace_samples               = ", trace_header['trace_samples'])
    print("sample_interval             = ", trace_header['sample_interval'])
    print("gain_type                   = ", trace_header['gain_type'])
    print("CDP_X                       = ", trace_header['CDP_X'])          
    print("CDP_Y                       = ", trace_header['CDP_Y'])          
    print("inline                      = ", trace_header['inline'])         
    print("xline                       = ", trace_header['xline'])          
    print("trace_unit                  = ", trace_header['trace_unit'])     
    print("inline_custom               = ", trace_header['inline_custom'])         
    print("xline_custom                = ", trace_header['xline_custom'])          


### Load SEGY
We will load the SEGY file from S3 and create a StreamingBody object we can use to stream data into the notebook.  We do not need to copy the file locally, we will stream data bit-by-bit as needed.  

You should modify these variables to point towards your own S3 bucket and SEGY file.  

In [ ]:
# Load in the 100GB SEGY data

source_bucket   = 'geophysics-on-cloud'                 # S3 bucket name with input data
source_folder   = 'poseidon/seismic/segy'               # Folder path
source_filename = 'psdn11_TbsdmF_full_w_AGC_Nov11.segy' # Filename

s3 = boto3.resource('s3')
segy_obj = s3.Object(source_bucket, f"{source_folder}/{source_filename}")
segy_stream = segy_obj.get()['Body']

### Read Headers
Lets read the text and binary headers of the SEGY file.  We are assuming the file follows the SEGY Rev1 standard, which is the case for this file.

In this format, the first 3200 bytes are the text header, the next 400 are the binary headers, so we are saving it to variables.

In [ ]:
text_header_raw = segy_stream.read(3200)
bin_header_raw = segy_stream.read(400)

text_header = DecodeTextHeader(text_header_raw)

print("Text Header:")
print(text_header)

bin_header = DecodeBinHeader(bin_header_raw)
print("\nBinary Header:")
PrintBinHeader(bin_header)

### Traces
After the file headers, the remainder of the SEGY consists of pairs of trace header blocks and the actual trace values in sequence.

Lets read one of the binary trace headers.  This dataset contains a handful of traces without amplitudes, so we will skip a few traces at first to check to see the data is being read correctly.

In [ ]:
# Skip a 1000 traces, as the first handful do not have any usable data.
skip_no = 1000

for x in range(0, skip_no):
    trace_header_raw = segy_stream.read(240)
    trace_header = DecodeTraceHeader(trace_header_raw)
    trace_raw = segy_stream.read(trace_header['trace_samples']*4)

trace_header_raw = segy_stream.read(240)
trace_header = DecodeTraceHeader(trace_header_raw)
PrintTraceHeaders(trace_header)

### Trace Data
SEGY stores the trace amplitudes in different formats, outlined by the binary header "data_sample_format".  

In this file the trace data is in IEEE 4-byte float, which is directly useable in Python.  We do not need to convert it, but only unpack it from the byte format.

In [ ]:
trace_raw = segy_stream.read(trace_header['trace_samples']*4)

trace = []
for x in range(4, trace_header['trace_samples']*4+4, 4):
    trace.append(struct.unpack('>f', trace_raw[x-4:x])[0])

# Lets plot the trace
limits = np.amax(np.absolute(trace))
plt.figure(figsize=(5, 10))
plt.plot(trace, range(trace_header['trace_samples']), 'red')
plt.xlim(-limits, limits)
plt.ylim(trace_header['trace_samples'], 0)
plt.axvline(0, c='grey')

In [ ]:
segy_stream.close()

### Process Data on EC2/SageMaker Notebook

First, for a benchmark, lets start processing here and see how long it would take on a single vCPU.

In [ ]:
segy_stream = segy_obj.get()['Body']

text_header_raw = segy_stream.read(3200)
bin_header_raw = segy_stream.read(400)
text_header = DecodeTextHeader(text_header_raw)
bin_header = DecodeBinHeader(bin_header_raw)

In [ ]:
trace_amp_mean = {}
start_time = time.time()

# Iterate through all the traces
while True:
    trace_header_raw = segy_stream.read(240)
    trace_header = DecodeTraceHeader(trace_header_raw)
    trace_raw = segy_stream.read(trace_header['trace_samples']*4)

    trace = []
    for x in range(4, trace_header['trace_samples']*4+4, 4):
        trace.append(struct.unpack('>f', trace_raw[x-4:x])[0])
    
    trace_amp_mean[trace_header['trace_seq_no_all']] = np.mean(np.absolute(trace))
    
    if trace_header['trace_seq_no_all']%1000 == 0:
        print("Trace #{} has a mean amplitude of {}, elapse time is {:.2f} seconds.".format(
                                        trace_header['trace_seq_no_all'], 
                                        trace_amp_mean[trace_header['trace_seq_no_all']], 
                                        time.time() - start_time))
        
    if trace_header['trace_seq_no_all'] > 5000:
        print("Stopping here.")
        break

segy_stream.close()

### Scale Out

Processing the SEGY file with a single machine is quite slow.  It would take about 4 hours to complete on this T2.Medium instance.  We could scale up and use a more powerful instance type, but then we are paying per hour, even if we are not using the full compute capacity.  Futhermore will will have to code the parallel processing.  It can quickly start becoming expensive to leave the machines running and the code complexity will increase.  The flow would look like the image below:


![title](images/Page-1.png)


So lets leverage the power of the cloud to scale out and distribute the processing workload with on-demand processing, Lambda!  Lets split this workload and send it to 1000 Lambdas that can run in parallel.  We do not need to send any seismic data to Lambda, we will simply tell each one where to find the file on S3 and which portion of the file to load.


![title](images/Page-2.png)


If you are following along with the GitHub repo and using the CloudFormation template, put in the bucket name you created when deploying the template.

There is a Lambda function that contains the same calculations we performed above called [SegyBatchProcessMeanAmp](https://console.aws.amazon.com/lambda/home?region=us-east-1#/functions/SegyBatchProcessMeanAmp).  If you want, go check it out in the AWS Console.  It was deployed with the CF template.
Lets get things setup up below.  Adjust variables below:

**lambda_name** - Visit the Lambda page in the AWS Console and see what the final name is for your. 

**results_bucket** - Rename to the S3 bucket you created with the CloudFormation template.

In [ ]:
lambda_name        = "Seismic-Lambdas-4-SegyMeanAmpLambda-15NCECNNVG6X2"    # Name of the Lambda function to invoke
results_bucket     = "seismic-results-vavourak-4"                           # Bucket to use
mean_amp_folder    = "temp-trace-bundles-poseidon-mean-amp"                 # Subfolder to place the calculation results
concurrent_lambdas = 1000                                                   # Number of Lambdas to invoke

Now lets split up the SEGY file into byte ranges and invoke 1000 Lambda functions.

In [ ]:
# Get S3 object
segy_obj = s3.Object(source_bucket, f"{source_folder}/{source_filename}")

# Define some needed variables based off the above parameters
start_time = time.time()
header_size = 3600
trace_header_size = 240
trace_size = bin_header['samples_per_trace'] * 4
trace_size_with_headers = trace_size + trace_header_size
filesize = segy_obj.content_length
trace_count = int((filesize - 3600) / trace_size_with_headers)
bundle_size = round(trace_count/concurrent_lambdas)
lambda_counter = 0

lambda_client = boto3.client('lambda')

results_file_list = [] # Lets keep track of the output file names, so we can grab them later

print(f"Total traces in file: {trace_count}")
print(f"Traces per Lambda for {concurrent_lambdas} concurrency (not rounded): {trace_count/concurrent_lambdas}")
print(f"Traces per Lambda, rounded up: {round(trace_count/concurrent_lambdas+0.5)}")

In [ ]:
# Send the trace bundle information over to Lambda
for bundle in range(0, int(trace_count), bundle_size):
    lambda_counter = lambda_counter + 1
    bytes_start = bundle * trace_size_with_headers + header_size
    bytes_stop = (bundle + bundle_size) * trace_size_with_headers + header_size - 1
    print(f"Lambda #{lambda_counter}, bundled traces: {bundle}-{bundle+bundle_size}, bytes: {bytes_start}-{bytes_stop}.")
    
    # Build the message for the Lambda to find the files
    payload = {
        "bucket_in"          : source_bucket,
        "folder_in"          : source_folder,
        "filename_in"        : source_filename,
        "bucket_out"         : results_bucket,
        "folder_out"         : mean_amp_folder,
        "bytes_start"        : bytes_start,
        "bytes_stop"         : bytes_stop,
        "use_custom_lines"   : 0,
        "data_sample_format" : bin_header['data_sample_format']
    }

    # Invoke the Lambda SegyBatchProcessMeanAmp
    response = lambda_client.invoke(FunctionName=lambda_name,
                                    InvocationType='Event',
                                    Payload=json.dumps(payload))

    results_file_list.append(f"{mean_amp_folder}/{source_filename}.{bytes_start}-{bytes_stop}.pkl")
    
print("Done!  Elapse time to gather traces and send to Lambda: {:0.2f} seconds, now waiting a bit for processing to complete.".format(time.time() - start_time))

time.sleep(200)      # Waiting before carrying on next steps, to allow time for Lambda to finish.

With a 1000 Lambdas, taking roughly 170 seconds each, 46 seconds to start them, our total time is about 4 minutes.  The cost to perform this calculation on this 100GB file is about $0.35.  As it takes 46 seconds to invoke the Lambdas, there is some room for optimization here to invoke them faster.  Having more Lambda concurrency available will help further, as each one can process a smaller portion of the file.  

### Load Results
The Lambdas should be done by now. Lets load in the results from S3.  Due to limitations in matplotlib's ability to display large datasets, we will only load 10% of the data.

In [ ]:
s3_client = boto3.client('s3')

traces = []
trace_mean_amp = []
trace_x = []
trace_y = []
start_time = time.time()

# Iterate through the files
#for x in range(0, len(results_file_list)):
for x in range(0, 1000, 10):
    print("Reading file: ", results_file_list[x])
    
    # Get file from S3, convert from Pickle format
    object = s3_client.get_object(Bucket=results_bucket, Key=results_file_list[x])
    serializedObject = object['Body'].read()
    trace_bundle_temp = pickle.loads(serializedObject)
    
    #for y in range(0, len(trace_bundle_temp)):
    #    traces.append(trace_bundle_temp[y])
    # Split the tuple [1,2,3] into seperate variables for easier use
    for y in range(0, len(trace_bundle_temp)):
        trace_mean_amp.append(trace_bundle_temp[y][0])
        trace_x.append(trace_bundle_temp[y][1])
        trace_y.append(trace_bundle_temp[y][2])

print("Number of traces loaded: {}, elapsed time: {:0.2f} seconds.".format(len(trace_mean_amp), time.time() - start_time))
print("Elapsed time: {:0.2f} seconds.".format(time.time() - start_time))


Well will cull any traces that lack data.  There are a handful that do not have any amplitude and will make it more complicated displaying them.

In [ ]:
trace_culled = []
trace_x_culled = []
trace_y_culled = []

print('Number of traces before:', len(trace_mean_amp))

# Go through every trace and cull any traces without an amplitude
counter = 0
for i in range(0, len(trace_mean_amp)):
    if trace_mean_amp[i] > 1000:
        trace_culled.append(trace_mean_amp[i])
        trace_x_culled.append(trace_x[i])
        trace_y_culled.append(trace_y[i])

print('Number of traces after: ', len(trace_culled))

Lets map out the mean amplitudes using a Matplotlib scatter plot.

In [ ]:
plt.figure(figsize=(12, 10))
plt.scatter(trace_x_culled, trace_y_culled, c=trace_culled)
plt.show()

### Conclusion
We have successfully performed a mathematical calculation on a 100GB seismic SEGY file without having to spin up or down a single computer.  All at the cost of $0.35 and 4 minutes of processing time.  This process is scalable and repeatable.  It can be expanded upon and automated with AWS Step Functions to orchestrate multiple steps and SageMaker to perform ML inference on the data.  Lambda concurrency can even be increased from the default 1000 to hundreds of thousands, greatly increasing performance.  

Of course, this calculation is not at the complexity of Full Waveform Inversion or other computations that require high performance computing (HPC).  However, if those calculations can be adjusted to work with Lambda (up to 6 vCPU, 10GB RAM, 15 minute limit), a huge amount of undifferentiated heavy lifting can be removed and offer an alternative to the complixity of scheduling thousands of EC2 spot instances.

### Cleanup

Lets clean up the files we generated, so that we can delete the CloudFormation stack without issue.

In [ ]:
for x in range(0, len(results_file_list)):
    _ = s3_client.delete_object(Bucket=results_bucket, Key=results_file_list[x])
    print("Deleting: {}".format(results_file_list[x]))